# How to Clean Sensitive Data You Can't Even See With Antigranular

## Motivation

## Setup

## 1. Removing duplicates under differential privacy

## 2. Imputing missing values under differential privacy

## 3. Fixing data types under DP

## 4. Filtering simple outliers under DP

## 5. Formatting dates under DP

## 6. Scaling or normalizing data under DP

## 7. Encoding categorical features for ML under DP

## Conclusion